In [1]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.9892, 0.0419, 0.5577],
        [0.4436, 0.6087, 0.4707],
        [0.1496, 0.0144, 0.6942],
        [0.5517, 0.3584, 0.3041],
        [0.1564, 0.0783, 0.6426]])


In [2]:
#Import dependencies
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
# Download training data from open datasets
#FashionMNIST dataset, which is a collection of images of clothing items like shirts, pants, and shoes
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [4]:
#Dataset stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset
#Define batch size i.e each element in the dataloader iterable will return a batch of 64 features and labels
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [5]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Create a class that inherits from nn.module. Define the layers of network in the __init__ function. To accelerate operations in NN we move it to GPU/MPS if available
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
#To train a model we need a loss function and an optimizer:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [7]:
#In a single training loop model makes predictions on training dataset and backpropagates the prediction error to adjust model's parameters
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
#Check the models performance against the test dataset to ensure it is learning
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.292591  [   64/60000]
loss: 2.296645  [ 6464/60000]
loss: 2.274471  [12864/60000]
loss: 2.266799  [19264/60000]
loss: 2.259984  [25664/60000]
loss: 2.203196  [32064/60000]
loss: 2.230847  [38464/60000]
loss: 2.180249  [44864/60000]
loss: 2.186100  [51264/60000]
loss: 2.158975  [57664/60000]
Test Error: 
 Accuracy: 37.2%, Avg loss: 2.151471 

Epoch 2
-------------------------------
loss: 2.158168  [   64/60000]
loss: 2.159994  [ 6464/60000]
loss: 2.095614  [12864/60000]
loss: 2.111493  [19264/60000]
loss: 2.071891  [25664/60000]
loss: 1.995161  [32064/60000]
loss: 2.043435  [38464/60000]
loss: 1.948828  [44864/60000]
loss: 1.959743  [51264/60000]
loss: 1.898062  [57664/60000]
Test Error: 
 Accuracy: 58.4%, Avg loss: 1.884827 

Epoch 3
-------------------------------
loss: 1.912913  [   64/60000]
loss: 1.895998  [ 6464/60000]
loss: 1.760966  [12864/60000]
loss: 1.810133  [19264/60000]
loss: 1.711381  [25664/60000]
loss: 1.648141  [32064/600

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [11]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>